In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import json

import windIO
import wind
import windCAD
import foam

## Building Geometry

In [2]:
caseDir = r"D:\tempData_depot\simData_CandC\wt_tngE2/"

tapFile = caseDir+"tngTapDefinition.csv"
data = pd.read_csv(tapFile)

faceID = [1,2,3,4,5,6]
name = ['Roof_1','Roof_2','Wall_1','Wall_2','Wall_3','Wall_4']
note = ['','','','','','']
origin = [[0,0,0.08],
          [0,-0.0915,0.07809375],
          [-0.1372,0,0],
          [0,-0.0915,0],
          [0.1372,0,0],
          [0,0.0915,0],]
basisVectors = [[[1,0,0],[0,0.999783057,0.020828814],[0,-0.020828814,0.999783057]],
                [[1,0,0],[0,0.999783057,-0.020828814],[0,0.020828814,0.999783057]],
                [[0,0,-1],[-1,0,0],[0,1,0]],
                [[1,0,0],[0,0,-1],[0,1,0]],
                [[0,0,1],[1,0,0],[0,1,0]],
                [[-1,0,0],[0,0,1],[0,1,0]],]
# origin_plt = [[0, 0],
#                 [0, -0.091519855],
#                 [0, -0.2332],
#                 [0, -0.18559375],
#                 [0, -0.2332],
#                 [0, -0.18559375],]
origin_plt = [[0, 0],
              [0, 0],
              [0, 0],
              [0, 0],
              [0, 0],
              [0, 0],
              ]
# basisVectors_plt = [[[0, -1], [1, 0]],
#                     [[0, -1],[1, 0]],
#                     [[1, 0], [0, 1]],
#                     [[0, -1], [1, 0]],
#                     [[-1, 0], [0, -1]],
#                     [[0, 1], [-1, 0]],]
# basisVectors_plt = [[[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     [[1, 0], [0, 1]],
#                     ]
basisVectors_plt = [[[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    [[0, 1], [-1, 0]],
                    ]
vertices = [[[-0.091519855,-0.1372],[0,-0.1372],[0,0.1372],[-0.091519855,0.1372],[-0.091519855,-0.1372]],
            [[0,-0.1372],[0.091519855,-0.1372],[0.091519855,0.1372],[0,0.1372],[0,-0.1372]],
            [[-0.0915,-0.2332],[0.0915,-0.2332],[0.0915,-0.15510625],[0,-0.1532],[-0.0915,-0.15510625],[-0.0915,-0.2332]],
            [[0.107519855,-0.1372],[0.185613605,-0.1372],[0.185613605,0.1372],[0.107519855,0.1372],[0.107519855,-0.1372]],
            [[-0.0915,0.2332],[0.0915,0.2332],[0.0915,0.15510625],[0,0.1532],[-0.0915,0.15510625],[-0.0915,0.2332]],
            [[-0.107519855,-0.1372],[-0.185613605,-0.1372],[-0.185613605,0.1372],[-0.107519855,0.1372],[-0.107519855,-0.1372]],]

H = 0.08
B = 0.2744
D = 0.183

In [3]:
# Zone Dict definitions

zoneDict = [
        {    # Face 1
            0:['NBCC', 'Zone c', np.array([[-0.091519855,-0.1372],[-0.071519855,-0.1372],[-0.071519855,-0.1172],[-0.091519855,-0.1172],[-0.091519855,-0.1372]])],
            1:['NBCC', 'Zone c', np.array([[-0.091519855,0.1172],[-0.071519855,0.1172],[-0.071519855,0.1372],[-0.091519855,0.1372],[-0.091519855,0.1172]])],
            2:['NBCC', 'Zone s', np.array([[-0.071519855,-0.1372],[0,-0.1372],[0,-0.1172],[-0.071519855,-0.1172],[-0.071519855,-0.1372]])],
            3:['NBCC', 'Zone s', np.array([[-0.091519855,-0.1172],[-0.071519855,-0.1172],[-0.071519855,0.1172],[-0.091519855,0.1172],[-0.091519855,-0.1172]])],
            4:['NBCC', 'Zone s', np.array([[-0.071519855,0.1172],[0,0.1172],[0,0.1372],[-0.071519855,0.1372],[-0.071519855,0.1172]])],
            5:['NBCC', 'Zone r', np.array([[-0.071519855,-0.1172],[0,-0.1172],[0,0.1172],[-0.071519855,0.1172],[-0.071519855,-0.1172]])],
        },
        {   # Face 2
            0:['NBCC', 'Zone c', np.array([[0.091519855,-0.1372],[0.071519855,-0.1372],[0.071519855,-0.1172],[0.091519855,-0.1172],[0.091519855,-0.1372]])],
            1:['NBCC', 'Zone c', np.array([[0.091519855,0.1172],[0.071519855,0.1172],[0.071519855,0.1372],[0.091519855,0.1372],[0.091519855,0.1172]])],
            2:['NBCC', 'Zone s', np.array([[0.071519855,-0.1372],[0,-0.1372],[0,-0.1172],[0.071519855,-0.1172],[0.071519855,-0.1372]])],
            3:['NBCC', 'Zone s', np.array([[0.091519855,-0.1172],[0.071519855,-0.1172],[0.071519855,0.1172],[0.091519855,0.1172],[0.091519855,-0.1172]])],
            4:['NBCC', 'Zone s', np.array([[0.071519855,0.1172],[0,0.1172],[0,0.1372],[0.071519855,0.1372],[0.071519855,0.1172]])],
            5:['NBCC', 'Zone r', np.array([[0.071519855,-0.1172],[0,-0.1172],[0,0.1172],[0.071519855,0.1172],[0.071519855,-0.1172]])],
        },
        {   # Face 3
            0:['NBCC', 'Zone e', np.array([[-0.0915,-0.2332],[-0.0715,-0.2332],[-0.0715,-0.154689583],[-0.0915,-0.15510625],[-0.0915,-0.2332]])],
            1:['NBCC', 'Zone e', np.array([[0.0715,-0.2332],[0.0915,-0.2332],[0.0915,-0.15510625],[0.0715,-0.154689583],[0.0715,-0.2332]])],
            2:['NBCC', 'Zone w', np.array([[-0.0715,-0.2332],[0.0715,-0.2332],[0.0715,-0.154689583],[0,-0.1532],[-0.0715,-0.154689583],[-0.0715,-0.2332]])],
        },
        {   # Face 4
            0:['NBCC', 'Zone e', np.array([[0.107519855,-0.1372],[0.185613605,-0.1372],[0.185613605,-0.1172],[0.107519855,-0.1172],[0.107519855,-0.1372]])],
            1:['NBCC', 'Zone e', np.array([[0.107519855,0.1172],[0.185613605,0.1172],[0.185613605,0.1372],[0.107519855,0.1372],[0.107519855,0.1172]])],
            2:['NBCC', 'Zone w', np.array([[0.107519855,-0.1172],[0.185613605,-0.1172],[0.185613605,0.1172],[0.107519855,0.1172],[0.107519855,-0.1172]])],
        },
        {   # Face 5
            0:['NBCC', 'Zone e', np.array([[-0.0915,0.2332],[-0.0715,0.2332],[-0.0715,0.154689583],[-0.0915,0.15510625],[-0.0915,0.2332]])],
            1:['NBCC', 'Zone e', np.array([[0.0715,0.2332],[0.0915,0.2332],[0.0915,0.15510625],[0.0715,0.154689583],[0.0715,0.2332]])],
            2:['NBCC', 'Zone w', np.array([[-0.0715,0.2332],[0.0715,0.2332],[0.0715,0.154689583],[0,0.1532],[-0.0715,0.154689583],[-0.0715,0.2332]])],
        },
        {   # Face 6
            0:['NBCC', 'Zone e', np.array([[-0.107519855,-0.1372],[-0.185613605,-0.1372],[-0.185613605,-0.1172],[-0.107519855,-0.1172],[-0.107519855,-0.1372]])],
            1:['NBCC', 'Zone e', np.array([[-0.107519855,0.1172],[-0.185613605,0.1172],[-0.185613605,0.1372],[-0.107519855,0.1372],[-0.107519855,0.1172]])],
            2:['NBCC', 'Zone w', np.array([[-0.107519855,-0.1172],[-0.185613605,-0.1172],[-0.185613605,0.1172],[-0.107519855,0.1172],[-0.107519855,-0.1172]])],
        },
]

# file = caseDir+'NBCC_zoneDict.json'
# with open(file, 'w') as f:
#     json.dump(zoneDict,f, indent=4, separators=(',', ':'))

In [ ]:

faces = []
for i,f in enumerate(faceID):
    idx = data.index[data.faceID == f]
    tapCoords = np.transpose(np.array([data.x[idx], data.y[idx]]))
    tapNos = np.array(data.tapNo[idx],dtype=int)
    idxOrig = idx

    fc = windCAD.face(
                name=name[i],
                ID=f,
                origin=origin[i],
                basisVectors=basisVectors[i],
                origin_plt=origin_plt[i],
                basisVectors_plt=basisVectors_plt[i],
                vertices=vertices[i],
                tapNo=tapNos,
                tapIdx=idxOrig,
                tapName=None,
                tapCoord=tapCoords,
                zoneDict=zoneDict[i],
                # nominalPanelAreas=[1.0e-4, 2.45e-4, 4.8e-4, 7.1e-4, 1e-3], #[5e-5, 8e-5, 1.5e-4, 5e-4],
                nominalPanelAreas=[1.5e-5,1.0e-4, 4.8e-4, 7.1e-4, 1e-3], 
                # nominalPanelAreas=[4.8e-4, 7.2e-4, 1e-3],
                numOfNominalPanelAreas=3,
                )
    # fc.plot(figSize=[20,15], overlayPanels=True, overlayTaps=True, overlayTribs=True, overlayZones=True)
    faces.append(fc)

# file = caseDir+'ttu_bldgGeom.json'
# allFaces.writeToFile(file_basic=file)

In [6]:
ttu = wind.bldgCp(bldgName='TTU',
                faces=faces,
                H=H,D=D,B=B,roofSlope=1.2,lScl=0.02,
                caseName='TNG-E2R1',
                )
print(ttu.error_in_panels)
print(ttu.error_in_zones)

[(([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], [])), (([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], [])), (([], [], [], [], []), ([], [], [], [], []), ([32, 53, 675, 692], [], [], [], [])), (([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], [])), (([], [], [], [], []), ([], [], [], [], []), ([31, 32, 674, 707], [], [], [], [])), (([], [], [], [], []), ([], [], [], [], []), ([], [], [], [], []))]
[((1, 2, 3, 4), (1, 2, 3, 4), (1, 2, 3, 4), (), (1, 2, 3, 4), ()), ((1, 2, 3, 4), (1, 2, 3, 4), (1, 2, 3, 4), (), (1, 2, 3, 4), (0, 1)), ((1, 2, 3, 4), (2, 3, 4), ()), ((1, 2, 3, 4), (1, 2, 3, 4), (3,)), ((1, 2, 3, 4), (2, 3, 4), (2,)), ((1, 2, 3, 4), (1, 2, 3, 4), ())]


d:\OneDrive - The University of Western Ontario\Documents\PhD\Thesis\CodeRepositories\windCalc\src\windCAD.py:464: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  num[a] += len(self.panels[z][a])


In [ ]:
fig = plt.figure(figsize=[10,8])
ax = fig.add_subplot()

ttu.plotEdges(ax=ax)
ttu.plotTaps(ax=ax)

ax.axis('equal')
ax.axis('off')



## Wind field

In [ ]:
dt = 3.7890e-04

file = r'D:\tempData_depot\simData_CandC\tngE2\tngE2Fr1_profile.csv'
temp = pd.read_csv(file)

file = caseDir+'tngE2Fr1_TH-UofT.npy'
U_TH = np.load(file)
file = caseDir+'tngE2Fr1_TH-VofT.npy'
V_TH = np.load(file)
file = caseDir+'tngE2Fr1_TH-WofT.npy'
W_TH = np.load(file)


In [ ]:

vel = wind.profile(name="BLWT-E2", Z=temp.Z,UofT=U_TH,VofT=V_TH,WofT=W_TH,H=H,dt=dt,nSpectAvg=64)
vel.plot()

## BLWT $C_p$ data

In [ ]:
mainRefPitotChnlIdx = 2
fps2mps = 0.3048
Zpitot = 1.48
Ntaps = 456

dataDirHFPI = r'E:\TNG\rawHFPI\tngE2p1/'

N_AoA = 3
# AoAids = [str(i).zfill(3) for i in range(1,N_AoA+1)]
AoAids = ['001', '010', '019']

AoA = np.zeros((N_AoA))
sampleRate = np.zeros((N_AoA))

for i,a in enumerate(AoAids):
    file_pssd = dataDirHFPI+'TNGp1E02R001P'+ AoAids[i] +'a.pssd'
    file_pssr = dataDirHFPI+'TNGp1E02R001P'+ AoAids[i] +'a.pssr'

    cp_data,analog,WTTDATALOG = windIO.readPSSfile(file_pssr,file_pssd)

    if i == 0:
        N_t = np.shape(cp_data)[0]
        CpTH = np.zeros((N_AoA,Ntaps,N_t)) # [N_AoA,Ntaps,Ntime]
        UpitotOfT = np.zeros((N_AoA,N_t))
    CpTH[i,:,:] = np.transpose(cp_data[:,0:Ntaps])
    UpitotOfT[i,:] = 29.917 * np.sqrt(analog[:,mainRefPitotChnlIdx]) * fps2mps
    AoA[i] = np.round(WTTDATALOG["APPSPE"][0][0][0][0][0][0][0][16][0][0],1)
    sampleRate[i] = WTTDATALOG["APPSPE"][0][0][0][0][0][0][0][12][0][0]
Upitot = np.mean(UpitotOfT,axis=1)



In [ ]:
ttu = wind.bldgCp(bldgName='TTU',
                faces=faces,
                H=H,D=D,B=B,roofSlope=1.2,lScl=0.02,
                caseName='TNG-E2R1',
                refProfile=vel,
                Zref_input=Zpitot,  # for the Cp TH being input below
                Uref_input=Upitot,  # for the Cp TH being input below
                samplingFreq=sampleRate[0],
                airDensity=1.125,
                AoA=AoA,
                CpOfT=CpTH,  # Cp TH referenced to Uref at Zref
                badTaps=None, # tap numbers to remove
                reReferenceCpToH=True, # whether or not to re-reference Cp building height
                pOfT=None,
                p0ofT=None,
                CpStats=None,
                peakMethod='minmax',
                )


In [ ]:
fig = plt.figure(figsize=[20,15])
ax = fig.add_subplot()
ttu.plotZones(ax=ax)
ax.axis('equal')

In [ ]:
f = 1
fc = ttu[f]
a = 0
d = 1
fldRange=[-2,1]

fig = plt.figure(figsize=[20,15])
ax = fig.add_subplot()
ttu.plotPanelCpStats(fieldName='mean',dxnIdx=d,aIdx=a,showValueText=False,ax=ax,fldRange=fldRange)
ttu.plotPanels(ax=ax,aIdx=a)
ttu.plotEdges(ax=ax,showName=False)
ttu.plotZones(ax=ax)
# ttu.plotTaps(ax=ax,showTapNo=True,fontsize=10)
# ttu.plotTribs(ax=ax)
ax.axis('equal')


In [ ]:

print(ttu.error_in_panels)
print(ttu.error_in_zones)
for i in range(5):
    print(f"A: {np.shape(ttu.panelAreas[i][2])}, \tGCp:{np.shape(ttu.CpStatsAreaAvgByZone[i][2]['mean'])}")

In [ ]:
i = 2
plt.figure()
plt.semilogx(ttu.panelAreas[i][2], ttu.CpStatsAreaAvgByZone[i][2]['peakMin'][0,:],'.k')
plt.show()

# LES

In [ ]:
lesCase = r'D:\tempData_depot\simData_CandC\ttuE087_900.0/'

## Wind field

In [ ]:

lesProf = foam.processVelProfile(caseDir=lesCase, probeName='probes.V1',H=H, trimTimeSegs=[[0,1.0]])



In [ ]:
p0Idx = 30
_, _, p0OfT = foam.readProbe(probeName='probes.V0', postProcDir=lesCase+'postProcessing/', field='p', trimTimeSegs=[[0,1.0]],)

pOfTfile = lesCase+'postProcessing/wallPressure_VTK/all_pOfT.npy'
pOfT = np.load(pOfTfile)
m,n = np.shape(pOfT)
pOfT = np.reshape(pOfT,(1,m,n))
print(np.shape(pOfT))

p0OfT = np.reshape(np.transpose(p0OfT)[p0Idx,:n],(1,-1))
print(np.shape(p0OfT))


In [ ]:



ttu = wind.bldgCp(bldgName='TTU',
                faces=faces,
                H=H,D=D,B=B,roofSlope=1.2,lScl=0.02,
                caseName='LES-E087-90deg',
                refProfile=lesProf,
                Zref_input=H,  # for the Cp TH being input below
                Uref_input=lesProf.Uh,  # for the Cp TH being input below
                samplingFreq=lesProf.samplingFreq,
                airDensity=1.125,
                AoA=[90.0,],
                # CpOfT=CpTH,  # Cp TH referenced to Uref at Zref
                # badTaps=None, # tap numbers to remove
                # reReferenceCpToH=True, # whether or not to re-reference Cp building height
                pOfT=pOfT,
                p0ofT=p0OfT,
                # CpStats=None,
                peakMethod='minmax',
                )

In [ ]:
f = 1
fc = ttu[f]
a = 2
d = 0
fldRange=[-2,1]

fig = plt.figure(figsize=[20,15])
ax = fig.add_subplot()
# ttu.plotPanelCpStats(fieldName='mean',dxnIdx=d,aIdx=a,showValueText=False,ax=ax,fldRange=fldRange)
ttu.plotPanels(ax=ax,aIdx=a)
# ttu.plotTapField(ax=ax,field='mean')
ttu.plotEdges(ax=ax,showName=False)
ttu.plotZones(ax=ax)
# ttu.plotTaps(ax=ax,showTapNo=True,fontsize=10)
# ttu.plotTribs(ax=ax)
ax.axis('equal')


In [ ]:
print((ttu.CpStats['std']))

# CAD lab

In [37]:
from shapely.geometry import MultiPolygon, Polygon
import sfepy
from sfepy.base.base import output

def generate_mesh(polygon, nominal_area):
    # Define the domain
    domain = sfepy.fem.domain.FEDomain('domain', dim=2)
    domain.create_poly_line(polygon, is_closed=True, epsilon=1e-2)
    domain.create_mesh(mesh_size=nominal_area**0.5, mesh_type='gmsh')
    return domain.mesh

def visualize_mesh(mesh):
    # Extract x and y coordinates of the nodes
    x, y = mesh.coors[:,0], mesh.coors[:,1]
    # Plot the nodes
    plt.scatter(x, y)
    # Plot the elements
    for el in mesh.elements:
        plt.plot(x[el], y[el])
    # Show the plot
    plt.show()


polygon = ttu[0].zoneDict[0][2].tolist()
A = ttu[0].nominalPanelAreas[0]
print(polygon)

# Generate the mesh
mesh = generate_mesh(polygon, A)

visualize_mesh(mesh)

[[-0.091519855, -0.1372], [-0.071519855, -0.1372], [-0.071519855, -0.1172], [-0.091519855, -0.1172], [-0.091519855, -0.1372]]


Exception: Could not get last error